In [9]:
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPool2D
from keras.utils import to_categorical

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

### Data preprocess

In [39]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

px.imshow(np.hstack(X_train[:10])).update_coloraxes(showscale=False).show()

w, h = X_train[0].shape
print(f"Single image shape is {w} x {h}")

Single image shape is 28 x 28


Convert target to binary array

In [40]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

Normalize features

In [41]:
max_val = np.max(X_train)
X_train_scaled = X_train / max_val
X_test_scaled = X_test / max_val

### Sequential with dense layers only (MLP)

In [13]:
model = Sequential([
    Input((w, h, 1)),
    Flatten(),
    Dense(128, activation='sigmoid'),
    Dense(128, activation='sigmoid'),
    Dense(10, activation='softmax'),
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,282 (462.04 KB)

 Trainable params: 118,282 (462.04 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
history = model.fit(X_train_scaled, y_train_cat, epochs=10, validation_data=(X_test_scaled, y_test_cat))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7096 - loss: 0.8648 - val_accuracy: 0.8376 - val_loss: 0.4458
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8545 - loss: 0.4015 - val_accuracy: 0.8547 - val_loss: 0.3990
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8740 - loss: 0.3483 - val_accuracy: 0.8630 - val_loss: 0.3745
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8790 - loss: 0.3308 - val_accuracy: 0.8602 - val_loss: 0.3851
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8870 - loss: 0.3047 - val_accuracy: 0.8740 - val_loss: 0.3494
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8930 - loss: 0.2916 - val_accuracy: 0.8748 - val_loss: 0.3463
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8973 - loss: 0.2769 - val_accuracy: 0.8795 - val_loss: 0.3356
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9010 - loss: 0.2675 - 

In [16]:
accuracy_history = pd.DataFrame({
    'train': history.history['accuracy'],
    'test': history.history['val_accuracy']
})

loss_history = pd.DataFrame({
    'train': history.history['loss'],
    'test': history.history['val_loss']
})

px.line(accuracy_history, title='Accuracy').show()
px.line(loss_history, title='Loss').show()

### Sequential with conv and pooling layers 

In [42]:
model = Sequential([
    Input((w, h, 1)),
    Conv2D(filters=16, kernel_size=3, strides=1, padding='same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=32, kernel_size=3, strides=1, padding='same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=3, strides=1, padding='same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(64, activation='elu'),
    Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 14, 14, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 14, 14, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 7, 7, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,874 (237.79 KB)

 Trainable params: 60,874 (237.79 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
history = model.fit(X_train_scaled, y_train_cat, epochs=10, batch_size=64, validation_data=(X_test_scaled, y_test_cat))

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.7536 - loss: 0.6872 - val_accuracy: 0.8640 - val_loss: 0.3908
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8774 - loss: 0.3445 - val_accuracy: 0.8835 - val_loss: 0.3169
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8979 - loss: 0.2829 - val_accuracy: 0.8933 - val_loss: 0.2946
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9064 - loss: 0.2555 - val_accuracy: 0.8967 - val_loss: 0.2803
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9153 - loss: 0.2298 - val_accuracy: 0.8981 - val_loss: 0.2751
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9238 - loss: 0.2057 - val_accuracy: 0.9073 - val_loss: 0.2625
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9285 - loss: 0.1905 - val_accuracy: 0.8982 - val_loss: 0.2825
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9342 - loss: 0.1775 - val_accuracy: 0.

In [20]:
accuracy_history = pd.DataFrame({
    'train': history.history['accuracy'],
    'test': history.history['val_accuracy']
})

loss_history = pd.DataFrame({
    'train': history.history['loss'],
    'test': history.history['val_loss']
})

px.line(accuracy_history, title='Accuracy').show()
px.line(loss_history, title='Loss').show()

### LeNet (for ditits MNIST)

<img src='images/LeNet.png' width=500px>

In [32]:
model = Sequential([
    Input((w, h, 1)),
    Conv2D(filters=6, kernel_size=5, strides=1, padding='valid'),
    MaxPool2D(),
    Conv2D(filters=16, kernel_size=5, strides=1, padding='valid'),
    MaxPool2D(),
    Flatten(),
    Dense(120, activation='sigmoid'),
    Dense(84, activation='sigmoid'),
    Dense(10, activation='softmax'),
])

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 24, 24, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 12, 12, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 8, 8, 16)       │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 4, 4, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 120)            │        30,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,426 (173.54 KB)

 Trainable params: 44,426 (173.54 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [38]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

w, h = X_train[0].shape

max_val = X_train.max()
X_train_scaled = X_train / max_val
X_test_scaled = X_test / max_val

px.imshow(np.hstack(X_train_scaled[:10])).update_coloraxes(showscale=False).show()

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [35]:
history = model.fit(X_train_scaled, y_train_cat, epochs=10, validation_data=(X_test_scaled, y_test_cat))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.1879 - loss: 2.2720 - val_accuracy: 0.6763 - val_loss: 1.5468
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7574 - loss: 1.1690 - val_accuracy: 0.8792 - val_loss: 0.5479
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8874 - loss: 0.4920 - val_accuracy: 0.9162 - val_loss: 0.3444
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9202 - loss: 0.3242 - val_accuracy: 0.9342 - val_loss: 0.2576
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9319 - loss: 0.2570 - val_accuracy: 0.9435 - val_loss: 0.2131
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9452 - loss: 0.2071 - val_accuracy: 0.9519 - val_loss: 0.1803
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9518 - loss: 0.1782 - val_accuracy: 0.9577 - val_loss: 0.1515
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9593 - loss: 0.1523 - 

In [36]:
accuracy_history = pd.DataFrame({
    'train': history.history['accuracy'],
    'test': history.history['val_accuracy']
})

loss_history = pd.DataFrame({
    'train': history.history['loss'],
    'test': history.history['val_loss']
})

px.line(accuracy_history, title='Accuracy').show()
px.line(loss_history, title='Loss').show()